In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=adbfb150d08596a351ee2b316baf6c1f158d1ef5a526b3ce40512dba7a1edaef
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

In [3]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508A6/Train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508A6/Test.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(1165, 20)
  State  Account Length  Area Code  ... Intl Charge CustServ Calls  TARGET
0    KS             128        415  ...        2.70              1       0
1    OH             107        415  ...        3.70              1       0
2    NJ             137        415  ...        3.29              0       0
3    OH              84        408  ...        1.78              2       0
4    OK              75        415  ...        2.73              3       0

[

In [4]:

#Copy Train data excluding target
trainData_Copy = train_data.iloc[:, :-1].copy()
testData_Copy = test_data.iloc[:, :-1].copy()

testData_Copy.head()


,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,VT,50,415,yes,yes,26,307.1,94,52.21,289.4,78,24.60,174.9,109,7.87,8.0,3,2.16,0
1,UT,72,415,no,no,0,118.2,106,20.09,167.2,136,14.21,214.2,106,9.64,12.2,3,3.29,3
2,KS,130,510,no,no,0,154.0,95,26.18,205.9,106,17.50,233.7,75,10.52,12.9,1,3.48,1
3,NV,143,408,no,no,0,155.5,101,26.44,213.4,89,18.14,237.9,61,10.71,7.6,11,2.05,1
4,DE,89,510,yes,no,0,125.6,108,21.35,213.0,90,18.11,181.7,108,8.18,5.4,5,1.46,1


In [6]:
#List of Categorical Features
categoricalFeatures = ["State","Int'l Plan","VMail Plan"]

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
X_train.head()


,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [7]:
y_train = train_data.iloc[:, -1]
y_test = test_data.iloc[:, -1]
print(y_train.shape)
print(y_test.head())

(1165,)
0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64


In [33]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print(clf_cv_score.mean())



accuracy Score (testing) for Decision Tree:0.916799
Confusion Matrix after hypertuning for Decision Tree
[[1031   39]
 [  66  126]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1070
           1       0.76      0.66      0.71       192

    accuracy                           0.92      1262
   macro avg       0.85      0.81      0.83      1262
weighted avg       0.91      0.92      0.91      1262

0.7880198019801979


In [38]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,150,10),'max_depth': range(1,50,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print(clf_cv_score.mean())

{'min_samples_split': 30, 'max_depth': 25}
accuracy Score (testing) for Decision Tree:0.919968
Confusion Matrix after hypertuning for Decision Tree
[[1045   25]
 [  76  116]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1070
           1       0.82      0.60      0.70       192

    accuracy                           0.92      1262
   macro avg       0.88      0.79      0.83      1262
weighted avg       0.92      0.92      0.91      1262

0.8313407244085752


In [35]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) for Decision Tree:{0:6f}".format(rfc.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(rfc_cv_score.mean())

accuracy Score (testing) for Decision Tree:0.898574
Confusion Matrix after hypertuning for Decision Tree
[[1066    4]
 [ 124   68]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      1.00      0.94      1070
           1       0.94      0.35      0.52       192

    accuracy                           0.90      1262
   macro avg       0.92      0.68      0.73      1262
weighted avg       0.90      0.90      0.88      1262

0.8744555842138835


In [40]:
#Hyperparameter tuning for random forest
parameters={'n_estimators': range(100,150,10),'min_samples_split' : range(20,150,10),'max_depth': range(20,50,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) for Decision Tree:{0:6f}".format(rfc.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(rfc_cv_score.mean())

{'n_estimators': 120, 'min_samples_split': 20, 'max_depth': 36}
accuracy Score (testing) for Decision Tree:0.888273
Confusion Matrix after hypertuning for Decision Tree
[[1069    1]
 [ 140   52]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1070
           1       0.98      0.27      0.42       192

    accuracy                           0.89      1262
   macro avg       0.93      0.63      0.68      1262
weighted avg       0.90      0.89      0.86      1262

0.8880688446995961


In [45]:
#SupportVector Machine=========================================================
svclassifier = LinearSVC()
svclassifier.fit(X_train,y_train)
sv_predict = svclassifier.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) for Decision Tree:{0:6f}".format(rfc.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,sv_predict))
print("=== Classification Report ===")
print(classification_report(y_test,sv_predict))

sv_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(sv_cv_score.mean())


accuracy Score (testing) for Decision Tree:0.888273
Confusion Matrix after hypertuning for Decision Tree
[[1070    0]
 [ 192    0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1070
           1       0.00      0.00      0.00       192

    accuracy                           0.85      1262
   macro avg       0.42      0.50      0.46      1262
weighted avg       0.72      0.85      0.78      1262

0.8870492091225929


In [48]:
#Hyperparameter tuning for SVM
parameters={'C': [1000, 1500, 2000, 2500],'max_iter':[1000,1100,1200]}
sv_random = RandomizedSearchCV(svclassifier,parameters,n_iter=15)
sv_random.fit(X_train,y_train)
grid_parm_sv=sv_random.best_params_
print(grid_parm_sv)
#contruct svm using the best parameters
sv= LinearSVC(**grid_parm_sv)
sv.fit(X_train,y_train)
sv_predict = sv.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) :{0:6f}".format(rfc.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,sv_predict))
print("=== Classification Report ===")
print(classification_report(y_test,sv_predict))

sv_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(sv_cv_score.mean())

{'max_iter': 1000, 'C': 1500}
accuracy Score (testing) for Decision Tree:0.888273
Confusion Matrix after hypertuning for Decision Tree
[[1070    0]
 [ 190    2]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1070
           1       1.00      0.01      0.02       192

    accuracy                           0.85      1262
   macro avg       0.92      0.51      0.47      1262
weighted avg       0.87      0.85      0.78      1262

0.8891543944310397


In [58]:
#Neural Network==========================================================
clfa = MLPClassifier()
clfa.fit(X_train, y_train)
clfa_predict=clfa.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) :{0:6f}".format(clfa.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clfa_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clfa_predict))

sv_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(sv_cv_score.mean())

accuracy Score (testing) :0.877971
Confusion Matrix after hypertuning for Decision Tree
[[1034   36]
 [ 118   74]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1070
           1       0.67      0.39      0.49       192

    accuracy                           0.88      1262
   macro avg       0.79      0.68      0.71      1262
weighted avg       0.86      0.88      0.86      1262

0.8527850348060015


In [60]:
#Hyperparameter tuning for Neural Network
parameters={'solver': ['sgd', 'adam'],'alpha': [0.0001, 0.05]}
clfa_random = RandomizedSearchCV(clfa,parameters,n_iter=15)
clfa_random.fit(X_train,y_train)
grid_parm_clfa=clfa_random.best_params_
print(grid_parm_clfa)
#contruct random forest using the best parameters
clfa= MLPClassifier(**grid_parm_clfa)
clfa.fit(X_train,y_train)
clfa_predict = clfa.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) :{0:6f}".format(clfa.score(X_test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clfa_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clfa_predict))

sv_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print(sv_cv_score.mean())

{'solver': 'sgd', 'alpha': 0.0001}
accuracy Score (testing) :0.871632
Confusion Matrix after hypertuning for Decision Tree
[[1066    4]
 [ 158   34]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1070
           1       0.89      0.18      0.30       192

    accuracy                           0.87      1262
   macro avg       0.88      0.59      0.61      1262
weighted avg       0.87      0.87      0.83      1262

0.8544467555999298


In [51]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [RandomForestClassifier(**grid_parm_rfc), DecisionTreeClassifier(**grid_parm),LinearSVC(**grid_parm_sv)]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.90753425]
    fold  1:  [0.89726027]
    fold  2:  [0.91065292]
    fold  3:  [0.91724138]
    ----
    MEAN:     [0.90817221] + [0.00720904]
    FULL:     [0.90815451]

model  1:     [DecisionTreeClassifier]
    fold  0:  [0.92123288]
    fold  1:  [0.91095890]
    fold  2:  [0.92439863]
    fold  3:  [0.89655172]
    ----
    MEAN:     [0.91328553] + [0.01086387]
    FULL:     [0.91330472]

model  2:     [LinearSVC]
    fold  0:  [0.86986301]
    fold  1:  [0.87328767]
    fold  2:  [0.87628866]
    fold  3:  [0.87931034]
    ----
    MEAN:     [0.87468742] + [0.00350605]
    FULL:     [0.87467811]



In [56]:
#STACKING - CONSTRUCT Random Forest MODEL==============================
model = RandomForestClassifier()
model = model.fit(S_Train, y_train)
y_pred = model.predict(S_Test)
#Hyperparameter tuning for random forest
parameters={'n_estimators': range(150,200,10),'min_samples_split' : range(70,120,10),'max_depth': range(30,70,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(S_Train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(S_Train,y_train)
rfc_predict = rfc.predict(S_Test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.

print("accuracy Score (testing) :{0:6f}".format(rfc.score(S_Test,y_test)))

print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

rfc_cv_score = cross_val_score(rfc, S_Train, y_train, cv=10, scoring="roc_auc")
print(rfc_cv_score.mean())

{'n_estimators': 160, 'min_samples_split': 100, 'max_depth': 64}
accuracy Score (testing) :0.919968
Confusion Matrix after hypertuning for Decision Tree
[[1059   11]
 [  90  102]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      1070
           1       0.90      0.53      0.67       192

    accuracy                           0.92      1262
   macro avg       0.91      0.76      0.81      1262
weighted avg       0.92      0.92      0.91      1262

0.8135576540847362
